In [ ]:
from google.colab import drive #ignorar si no se ejecuta en Colab
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install --upgrade jax jaxlib netket optax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.1/721.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found existing installation: jax 0.5.2
    Uninstalling jax-0.5.2:
      Successfully uninstalled jax-0.5.

In [ ]:


import os
import sys
import argparse
from typing import Sequence, Tuple, List
import time

import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt


import jax
import jax.numpy as jnp
import jaxlib
import flax
import flax.linen as nn
import optax


import netket as nk
import netket.experimental as nkx
from netket.sampler import MetropolisRule
from netket.utils.struct import dataclass
from netket.operator.spin import sigmax, sigmaz

In [ ]:
!jupyter nbconvert --to python "/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/GNN_Ansatz"
!jupyter nbconvert --to python "/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/Function_GNN"

[NbConvertApp] WARNING | pattern '/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/GNN_Ansatz' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equi

In [ ]:
sys.path.insert(0,'/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/GitHub_prep/')

In [ ]:
from utils import circulant, BestIterKeeper,InvertMagnetization
from hamiltonian import get_Hamiltonian,get_eigvals
from funciones_auxiliares import *
from mpgnn_ansatz import *
from Function_GNN import *

In [ ]:
print(jax.__version__)
print(jaxlib.__version__)
print(nk.__version__)
print(optax.__version__)
print(np.__version__)
print(flax.__version__)

0.5.3
0.5.3
3.18
0.2.5
2.0.2
0.10.6


In [ ]:


diagonalize = True
trivial = False
N = 20
alpha = 0.0 #3, 6
J = 1.0
b = 1
h=-1
num_message_pass = [2, 3, 4]

In [ ]:
# Creación del espacio de Hilbert y otros observables

hi = nk.hilbert.Spin(s=1 / 2, N=N)

#definimos métricas adicionales
renyi = nkx.observable.Renyi2EntanglementEntropy(
    hi, np.arange(0, N / 2 + 1, dtype=int)
)

mags = sum([(-1) ** i * sigmaz(hi, i) / N for i in range(N)])

magnet = sum([sigmaz(hi, i) / N for i in range(N)])


In [ ]:
# MC reglas de muestreo
rule1 = nk.sampler.rules.LocalRule() #flip de un solo spin para proponer nuevo estado
pflip = 1
sampler = nk.sampler.MetropolisSampler(
    hi, nk.sampler.rules.MultipleRules([rule1], [pflip])
)


In [ ]:
# Protocolo de entrenamiento
max_iters = 300
ramp_iter = 50
lrmax = 0.05


lr_schedule = optax.warmup_exponential_decay_schedule(
    init_value=1e-4,
    peak_value=0.05,
    warmup_steps=30,
    transition_steps=100,
    decay_rate=0.97
)


optimizer = nk.optimizer.Sgd(learning_rate = lr_schedule)

ds_schedule = optax.linear_schedule(1e-3, 1e-5, max_iters)
SR = nk.optimizer.SR(diag_shift=ds_schedule)


In [ ]:
order = 100

# Hamiltoniano
H ,N_norm  = get_Hamiltonian(N=N, J=J, alpha=alpha, return_norm=True)

with jax.default_device(jax.devices("cpu")[0]):
    Egs, eigenvec = get_eigvals(Hamiltonian=H, order=order, eigenvecs=True)




In [ ]:
soluciones = []
energias = []

for i in range(eigenvec.shape[1]):
    v = eigenvec[:, i]
    is_odd = jnp.allclose(v + v[::-1], 0.0, atol=1e-3)
    soluciones.append('No Trivial' if is_odd else 'Trivial')
    energias.append(Egs[i])

# visualizamos las irreps
plt.figure(figsize=(8, 4))
colors = ['red' if p == 'No Trivial' else 'blue' for p in soluciones]
plt.bar(range(len(energias)), energias, color=colors)
plt.xticks(range(len(energias)), soluciones, rotation=45)
plt.ylabel("Energía")
plt.title("Energía de los primeros autoestados por paridad")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
def get_adjacency_matrix(N):#al parecer en netket hay una funcion que te obtiene esto ya

    '''
    In: N number of spins
    Out: Adjacency matrix of the graph
    '''

    adjacency_matrix = jnp.eye(N, k=1) + jnp.eye(N, k=-1)

    adjacency_matrix = adjacency_matrix.at[0, -1].set(1)
    adjacency_matrix = adjacency_matrix.at[-1, 0].set(1)

    return adjacency_matrix

In [ ]:
def get_edge(matrix, N):  #Podemos obtener la matriz directamente aqui dentro
    '''
    In: -Adjacency matrix
    -Number of spins

    Out: List of edges (number_edges, 2)

    '''
    if matrix.ndim != 2:
        raise ValueError("La matriz debe ser de 2 dimensiones.")
    # jnp.argwhere devuelve las coordenadas donde la condición se cumple.
    edges = jnp.argwhere(matrix == 1, size=2*N)  # Specify size using N
    return edges

In [ ]:
#problema al intentar usar la funcion dentro del modelo, o paso un jnp.array como booleano o tengo que usar un np.array, problemas ambos
#Solución: pasar la lista de vecinos directamente al modelo ya que la lista de vecinos es fija.

def vecinos_por_nodo(N):
    """
    In: N number of spins

    Out: Dictionary of neighbors for each node. The neighbours are represented as the indexes they occupy in the get_edge array.

    """
    # Convertimos edges a un array de NumPy para evitar problemas con trazado de JAX.
    edges_np = np.array(get_edge(get_adjacency_matrix(N),N))
    num_nodes = get_adjacency_matrix(N).shape[0]
    vecinos_por_nodo = {}

    for i in range(num_nodes):
        # np.where retorna los índices donde se cumple la condición
        indices = np.where(edges_np[:, 0] == i)[0].tolist()
        vecinos_por_nodo[i] = indices

    return vecinos_por_nodo


In [ ]:
def suma_resultados_con_indices(resultados_mlp, node_features,vecinos_tuple):
    '''
    In:
      -resultados_mlp: messages
      -node_features: node features
      -vecinos_tuple: vecinos por nodo (el diccionario convertido a tupla)
        se usa vecinos para sólo construir el mensaje con la suma de nodos vecinos

    Out:
      -resultados_sumados: resultados de la suma de los mensajes de los vecinos


    '''

    resultados_sumados = jnp.zeros(node_features.shape[0])

    for nodo, indices_vecinos in vecinos_tuple:

        indices_vecinos_array = jnp.array(indices_vecinos)

        suma_vecinos = jnp.sum(resultados_mlp[indices_vecinos_array])  # Sumar resultados de vecinos indicados por los indices

        resultados_sumados = resultados_sumados.at[nodo].set(suma_vecinos)  # Asignar suma al nodo actual
    #print(type(resultados_sumados))
    return resultados_sumados

In [ ]:
def obtain_origin_goal(node_features, edge_features):
    '''
    In:
      Recibe array node_features (batch, N , 5) y edge features (num conexiones, 2)

    Out:
      origin_input (num conexiones, feature node origin)
      goal_input (num conexiones, feature node goal)

    '''
    origin_input=[]
    goal_input=[]
    #print(node_features.shape)
    node_features_spin = node_features[:,:,0]

    for i in range(len(edge_features)):#con ayuda de las edge_features, que nos dicen tambien como se relacionan los nodos obtenemos las listas de origin, goal
      origin=edge_features[i][0]
      goal=edge_features[i][1]
      origin_input.append(node_features_spin[:,origin]) # para cada batch obtenemos los node_features del origen de cada conexion
      goal_input.append(node_features_spin[:,goal]) # lo mismo para el destino

    origin_input=jnp.array(origin_input)
    goal_input=jnp.array(goal_input)

    #print(origin_input.shape)
    #print(goal_input.shape)

    origin_input = jnp.stack(origin_input, axis=1)
    goal_input   = jnp.stack(goal_input, axis=1)

    return origin_input, goal_input

In [ ]:
def get_exchange_matrix(N_norm, b, N, alpha, J):
  '''
  Pasamos todo lo necesario para obtener los coeficientes de intercambio del Hamiltoniano.
  Construimos la matriz con los coeficientes de intercambio de los espines i, j.
  Queremos una matriz que nos muestre estos en la diagonal debera de estar b.
  '''
  coefficient_matrix = np.zeros((N, N))
  #calculamos las distancias
  for i in range(N):
    for j in range(N):
      if i!=j:
        distancia = jnp.min(jnp.array([jnp.abs(i-j), N-jnp.abs(i-j)])) # bboundary conditions
        distancia = distancia.astype(float) **(-alpha)
        coefficient_matrix[i,j] = distancia

      else:
        distancia = b
        coefficient_matrix[i,j] = distancia

  coefficient_matrix = J/N_norm * coefficient_matrix

  coefficient_matrix = jnp.array(coefficient_matrix)

  return coefficient_matrix

In [ ]:
# @title
def get_exchange_coefficients(edges, exchange_matrix):
  '''
  Le pasamos las conexiones y la matriz de intercambio, y selecciona solo los coeficientes de intercambio de las conexiones.
  Si i y j estan conectados nod devuelve pues J_{ij}.

  Funcion que nos da los coeficientes de intercambio del Hamiltoniano para cada una de las conexiones obtenidas en 'get_edges'
  '''
  exchange_coefficient = jnp.zeros(edges.shape[0])

  # Using jax.vmap for efficient vectorized operation
  def get_coefficient(edge):
    return exchange_matrix[edge[0], edge[1]]

  exchange_coefficient = jax.vmap(get_coefficient)(edges)
  return exchange_coefficient

In [ ]:
def obtener_vecinos_y_coeficientes(N, N_norm, b, alpha, J):
    '''

    Obtenemos los vecinos de cada nodo, y los coeficientes de intercambio de cada conexion.
    '''
    vecinos = vecinos_por_nodo(N)
    vecinos_hashable = tuple((k, tuple(v)) for k, v in sorted(vecinos.items()))

    coefficient_matrix = get_exchange_matrix(N_norm=N_norm, b=b, N=N, alpha=alpha, J=J)
    exchange_coefficients = get_exchange_coefficients(get_edge(get_adjacency_matrix(N),N),coefficient_matrix)
    exchange_coefficients_tuple = tuple(exchange_coefficients.tolist())

    return vecinos_hashable, exchange_coefficients_tuple

In [ ]:

# posibilidad para update, no se usa ahora mismo
class MLPUpdateBlock(nn.Module):
    '''
    Funcion entrenable que mezcla la información nodal que tenía con los mensajes construidos.
    '''

    hidden_dim: int
    dtype: jnp.dtype = jnp.float64

    @nn.compact
    def __call__(self, node_features, messages):
        messages_expanded = jnp.expand_dims(messages, axis=-1)
        x = jnp.concatenate([node_features, messages_expanded], axis=-1)

        x = nn.Dense(self.hidden_dim, dtype=self.dtype)(x)
        x = nn.swish(x)
        x = nn.Dense(node_features.shape[-1], dtype=self.dtype)(x)

        return x


In [ ]:
class MlpBlock(nn.Module):
    mlp_dim: int
    dtype: jnp.dtype = jnp.float32

    @nn.compact
    def __call__(self, x):
        y = nn.Dense(self.mlp_dim, dtype=self.dtype)(x)
        y = nn.swish(y)
        return nn.Dense(x.shape[-1], dtype=self.dtype)(y)

In [ ]:
class MLPMessage(nn.Module):
    features: Sequence[int] = (8, 5, 1)
    dtype: jnp.dtype = jnp.float32
    @nn.compact
    def __call__(self, node_features, edge_features, vecinos, exchange_coefficients):

        origin_input, goal_input = obtain_origin_goal(node_features, edge_features)

        batch = node_features.shape[0]
        edge_features_batch = jnp.broadcast_to(edge_features, (batch,) + edge_features.shape)
        exchange_coefficients_batch = jnp.broadcast_to(exchange_coefficients, (batch,) + exchange_coefficients.shape)

        origin_input = jnp.expand_dims(origin_input, axis=-1)
        goal_input = jnp.expand_dims(goal_input, axis=-1)
        exchange_coefficients_batch = jnp.expand_dims(exchange_coefficients_batch, axis=-1)

        x = jnp.concatenate([origin_input, goal_input, exchange_coefficients_batch], axis=-1)

        def forward_single(x_single, node_features_single):
            for feature in self.features:
                x_single = MlpBlock(feature, dtype=self.dtype)(x_single)
            messages = suma_resultados_con_indices(x_single, node_features_single, vecinos)
            return messages

        batched_forward = jax.vmap(forward_single, in_axes=(0, 0))
        return batched_forward(x, node_features)


In [ ]:
class GNNLayer(nn.Module):
    N: int
    dtype: jnp.dtype = jnp.float32

    @nn.compact
    def __call__(self, node_features, vecinos, exchange_coefficients):
        #calculamos la matriz de adjacencia y las conexiones (ahora edges)
        adj_matrix = get_adjacency_matrix(self.N)
        edges = get_edge(adj_matrix,self.N)

        messages = MLPMessage(dtype=self.dtype)(node_features, edges, vecinos, exchange_coefficients)# Trainable function

        #combinamos informacion de nodo original y mensajes
        combined_features = node_features.at[:, :, 0].add(messages) #combinamos sumando solo los parametros de spin (batch,N)

        updated_features = MLPMessage(dtype=self.dtype)(combined_features, edges, vecinos, exchange_coefficients)
        updated_features = combined_features.at[:, :, 0].add(updated_features)



        return updated_features

In [ ]:
class MPGNN(nn.Module):
    N: int
    num_message_pass: int
    vecinos: Tuple[Tuple[int, Tuple[int, ...]]] = None
    exchange_coefficients: Tuple = None
    node_components: int = 4
    dtype: jnp.dtype = jnp.float32

    @nn.compact
    def __call__(self, node_features):
        if node_features.ndim == 1:
            node_features = node_features[None, ...]

        #definimos los arrays de parametros entrenables
        node_components = self.node_components
        embedding = nn.Embed(node_features.shape[1], node_components, dtype=self.dtype) #(cuantos embeds, numero de componentes por embed)
        node_array = embedding(node_features)

        node_array_batched = jnp.broadcast_to(node_array, (node_features.shape[0], node_features.shape[1], self.node_components))

        #juntamos todos los node features
        node_features_expanded = jnp.expand_dims(node_features, axis=-1)
        node_features = jnp.concatenate([node_features_expanded, node_array_batched], axis=-1)

        vecinos = self.vecinos
        exchange_coefficients = self.exchange_coefficients
        exchange_coefficients = jnp.array(exchange_coefficients, dtype=self.dtype)

        for _ in range(self.num_message_pass):
            node_features = GNNLayer(self.N, dtype=self.dtype)(node_features, vecinos, exchange_coefficients)
            node_features = nn.swish(node_features)

        output = jnp.mean(node_features, axis=(1,2))
        return output


In [ ]:

from jax.scipy.special import logsumexp
class sym_MPGNN(nn.Module):
    trivial: bool
    N: int
    num_message_pass: int
    vecinos: Tuple[Tuple[int, Tuple[int, ...]]] = None
    exchange_coefficients: Tuple = None
    node_components: int = 4
    dtype: jnp.dtype = jnp.float32

    @nn.compact
    def __call__(self, node_features):
        model = MPGNN(
          N=self.N,
          num_message_pass = self.num_message_pass,
          vecinos=self.vecinos,
          exchange_coefficients =self.exchange_coefficients
      )
        output_x = model(node_features)
        output_inv_x = model(-1*node_features)
        if self.trivial:
            return logsumexp(jnp.array([output_x, output_inv_x]), axis = 0)
        else:
            weights = jnp.asarray([1., -1.])  # Ahora tiene forma (2, 1)
            weights = jnp.expand_dims(weights, axis=1)
            log_psi=logsumexp(jnp.stack([output_x+0.j, output_inv_x+0.j]), b=weights , axis=0)
            return log_psi

In [ ]:
def make_extract_metrics(metrics_history):
  '''
  Función que extrae metricas durante la optimización.
  '''
  def extract_metrics(step, log_data, driver):
      stats = driver.state.expect(H)
      energy = float(jnp.real(stats.mean))
      energy_error = float(jnp.real(stats.error_of_mean))

      loss = float(jnp.real(getattr(log_data[driver._loss_name], "mean")))
      variance = float(jnp.real(getattr(log_data[driver._loss_name], "variance")))

      metrics_history['step'].append(step)
      metrics_history['energy'].append(energy)
      metrics_history['energy_error'].append(energy_error)
      metrics_history['loss'].append(loss)
      metrics_history['variance'].append(variance)

      print(f"Step {step}: Energy = {energy:.6f} ± {energy_error:.2e}, Loss = {loss:.4f}, Variance = {variance:.4f}")
      return True

  return extract_metrics


In [ ]:
vecinos_hashable, exchange_coefficients_tuple = obtener_vecinos_y_coeficientes(N, N_norm, b, alpha, J)

In [ ]:
#definimos el modelo
modelos = []
for i in num_message_pass:

  model = sym_MPGNN(
      trivial=trivial,
      N=N,
      num_message_pass = i,
      vecinos=vecinos_hashable,
      exchange_coefficients =exchange_coefficients_tuple,
  )

  model_name = model.__class__.__name__
  modelos.append(model)



In [ ]:
import time

seed = int(time.time()) #creamos seed pseudoaleatoria para inicializar estado

In [ ]:
# Inicialización de estado de Montecarlo
vstate_list = []
gs_list = []
keeper_list = []
log_list = []


for i, model in enumerate(modelos):

  vstate = nk.vqs.MCState(
      sampler,
      model,
      n_samples=512,
      n_discard_per_chain=0,
      chunk_size=64,
      seed = seed
  )

  gs = nk.driver.VMC(
      H,
      optimizer,
      variational_state=vstate,
      preconditioner=SR,## stochastic reconfiguration
  )

  keeper = BestIterKeeper(H, N, 1e-8)
  log = nk.logging.RuntimeLog()

  #guardamos todas las inicializaciones, drivers y keepers uno por paso de mensaje
  vstate_list.append(vstate)
  gs_list.append(gs)
  keeper_list.append(keeper)
  log_list.append(log)


/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [ ]:
for i in vstate_list:
  n_params = sum(x.size for x in jax.tree_util.tree_leaves(i.parameters))
  print("Número total de parámetros:", n_params)

Número total de parámetros: 508
Número total de parámetros: 722
Número total de parámetros: 936


In [ ]:
#guardamos la primera energía, como se inicializa
vstate_init =  []
for vstate in vstate_list:
  vstate_init.append(vstate.expect(H))

/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [ ]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/first_energies.txt" # Ajustar el directorio
with  open(file_path, 'w') as f:
    for vstate in vstate_init:
        print(vstate, file=f)
    print("ground_energy=" ,Egs[0] , file=f)

In [ ]:
metrics_history = [{'step': [], 'energy': [], 'energy_error': [], 'loss': [], 'variance': []} for _ in range(len(num_message_pass))]

In [ ]:
# Running the VMC optimization
for i, gs in enumerate(gs_list):
    # Crea el callback específico para el i-ésimo modelo
    callback_fn = [keeper_list[i].update, make_extract_metrics(metrics_history[i])]

    gs.run(n_iter=max_iters, out = log_list[i], callback=callback_fn, show_progress=True)

  0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Step 0: Energy = -0.818716 ± 2.73e-03, Loss = -0.8187, Variance = 0.0038
Step 1: Energy = -0.823555 ± 2.64e-03, Loss = -0.8236, Variance = 0.0036
Step 2: Energy = -0.825391 ± 2.69e-03, Loss = -0.8254, Variance = 0.0037
Step 3: Energy = -0.823204 ± 2.77e-03, Loss = -0.8232, Variance = 0.0039
Step 4: Energy = -0.822407 ± 2.69e-03, Loss = -0.8224, Variance = 0.0037
Step 5: Energy = -0.827571 ± 2.70e-03, Loss = -0.8276, Variance = 0.0037
Step 6: Energy = -0.824498 ± 2.78e-03, Loss = -0.8245, Variance = 0.0040
Step 7: Energy = -0.821974 ± 2.90e-03, Loss = -0.8220, Variance = 0.0043
Step 8: Energy = -0.830381 ± 2.43e-03, Loss = -0.8304, Variance = 0.0030
Step 9: Energy = -0.831162 ± 2.57e-03, Loss = -0.8312, Variance = 0.0034
Step 10: Energy = -0.827812 ± 2.64e-03, Loss = -0.8278, Variance = 0.0036
Step 11: Energy = -0.825908 ± 2.62e-03, Loss = -0.8259, Variance = 0.0035
Step 12: Energy = -0.825380 ± 2.60e-03, Loss = -0.8254, Variance = 0.0035
Step 13: Energy = -0.824201 ± 2.65e-03, Loss = -

  0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/jax/_src/ops/scatter.py:93: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Step 0: Energy = -0.818124 ± 2.80e-03, Loss = -0.8181, Variance = 0.0040
Step 1: Energy = -0.825901 ± 2.58e-03, Loss = -0.8259, Variance = 0.0034
Step 2: Energy = -0.825213 ± 2.75e-03, Loss = -0.8252, Variance = 0.0039
Step 3: Energy = -0.822126 ± 2.86e-03, Loss = -0.8221, Variance = 0.0042
Step 4: Energy = -0.821339 ± 2.73e-03, Loss = -0.8213, Variance = 0.0038
Step 5: Energy = -0.826483 ± 2.77e-03, Loss = -0.8265, Variance = 0.0039
Step 6: Energy = -0.823864 ± 2.87e-03, Loss = -0.8239, Variance = 0.0042
Step 7: Energy = -0.823772 ± 2.86e-03, Loss = -0.8238, Variance = 0.0042
Step 8: Energy = -0.828013 ± 2.44e-03, Loss = -0.8280, Variance = 0.0030
Step 9: Energy = -0.831549 ± 2.48e-03, Loss = -0.8315, Variance = 0.0031
Step 10: Energy = -0.823690 ± 2.79e-03, Loss = -0.8237, Variance = 0.0040
Step 11: Energy = -0.826857 ± 2.50e-03, Loss = -0.8269, Variance = 0.0032
Step 12: Energy = -0.824351 ± 2.66e-03, Loss = -0.8244, Variance = 0.0036
Step 13: Energy = -0.823588 ± 2.61e-03, Loss = -

  0%|          | 0/300 [00:00<?, ?it/s]

Step 0: Energy = -0.819819 ± 2.75e-03, Loss = -0.8198, Variance = 0.0039
Step 1: Energy = -0.824743 ± 2.63e-03, Loss = -0.8247, Variance = 0.0036
Step 2: Energy = -0.822504 ± 3.05e-03, Loss = -0.8225, Variance = 0.0048
Step 3: Energy = -0.820137 ± 2.78e-03, Loss = -0.8201, Variance = 0.0039
Step 4: Energy = -0.817269 ± 2.97e-03, Loss = -0.8173, Variance = 0.0045
Step 5: Energy = -0.824135 ± 2.94e-03, Loss = -0.8241, Variance = 0.0044
Step 6: Energy = -0.825461 ± 2.83e-03, Loss = -0.8255, Variance = 0.0041
Step 7: Energy = -0.822731 ± 2.91e-03, Loss = -0.8227, Variance = 0.0043
Step 8: Energy = -0.829645 ± 2.34e-03, Loss = -0.8296, Variance = 0.0028
Step 9: Energy = -0.828228 ± 2.66e-03, Loss = -0.8282, Variance = 0.0036
Step 10: Energy = -0.825977 ± 2.73e-03, Loss = -0.8260, Variance = 0.0038
Step 11: Energy = -0.825909 ± 2.83e-03, Loss = -0.8259, Variance = 0.0041
Step 12: Energy = -0.826992 ± 2.58e-03, Loss = -0.8270, Variance = 0.0034
Step 13: Energy = -0.826914 ± 2.60e-03, Loss = -

In [ ]:
min_energy = keeper_list[0].best_energy
print(min_energy)

-0.8403563162421637


In [ ]:
metrics_history = np.array(metrics_history)

(3,)

In [ ]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/metrics_data.txt" # Ajustar el directorio

with open(file_path, 'w') as f:
    np.savetxt(f, metrics_history.T, fmt='%s', delimiter='\t')
    print(f"Metrics data saved to: {file_path}")

Metrics data saved to: /content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/metrics_data.txt


In [ ]:
energia_min_array = []
for i in range(len(num_message_pass)):
  energia_min_array.append(keeper_list[i].best_energy)

In [ ]:
energy_array = np.array(metrics_history[0]['energy'])
print(energy_array.shape)
relative_error = np.abs((energy_array - Egs[0]) / Egs[0])
print(relative_error.shape)

(300,)
(300,)


In [ ]:
min_rel_error = np.min(relative_error)
print(min_rel_error)

0.141779785328093


In [ ]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/evaluation_results.txt" # Ajustar el directorio
with open(file_path, 'w') as f:
    for keeper in keeper_list:
        if diagonalize:
            fidelity = np.abs(keeper.best_state.to_array().conj() @ eigenvec[:, 0])
            rel_err = np.abs((keeper.best_energy - Egs[0]) / Egs[0])
            print(f"Fidelity1: {fidelity:.5f}", file=f)
            print(f"Relative error in energy: {rel_err:.2E}", file=f)
        vsc = keeper.vscore
        print(f"V-score: {vsc:.2E}", file=f)
        S = np.real(keeper.best_state.expect(renyi).mean)
        print(f"Value for the Renyi-2 entropy: {S:.5f}", file=f)
        m = np.real(keeper.best_state.expect(magnet).mean)
        print(f"Value for the magnetization: {m:.5f}", file=f)
        ms = np.real(keeper.best_state.expect(mags).mean)
        print(f"Value for the staggered magnetization: {ms:.5f}", file=f)
        fluct = np.real(keeper.best_state.expect(magnet @ magnet).mean)
        print(f"Value for the squared magnetization: {fluct:.5f}", file=f)
        fluct_s = np.real(keeper.best_state.expect(mags @ mags).mean)
        print(f"Value for the squared staggered magnetization: {fluct_s:.5f}", file=f)
    print(f"Evaluation results saved to: {file_path}")

Evaluation results saved to: /content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/evaluation_results.txt


In [ ]:
import pandas as pd
import numpy as np

file_path = '/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/support.txt' # Ajustar el directorio
rows = []

j = 2
for keeper in keeper_list:
    for i in range(order):
        fidelity = np.abs(keeper.best_state.to_array().conj() @ eigenvec[:, i])
        rel_err = np.abs((keeper.best_energy - Egs[i]) / Egs[i])
        vsc = keeper.vscore

        rows.append({
            'MP': j,
            'Paridad': soluciones[i],
            'Energía': Egs[i],
            'Fidelidad': fidelity,
            'Error relativo': rel_err,
            'V-score': vsc
        })
    j += 1

# Crear DataFrame
df = pd.DataFrame(rows)

# Escribir como tabla alineada
with open(file_path, 'w') as f:
    f.write(df.to_string(index=False))


In [ ]:
import pandas as pd
import numpy as np

file_path = '/content/gdrive/MyDrive/Colab Notebooks/TFG/transformer_LR_WF/support.csv' # Ajustar el directorio
rows = []

j = 2
for keeper in keeper_list:
    for i in range(order):
        fidelity = np.abs(keeper.best_state.to_array().conj() @ eigenvec[:, i])
        rel_err = np.abs((keeper.best_energy - Egs[i]) / Egs[i])
        vsc = keeper.vscore

        rows.append({
            'MP': j,
            'Paridad': soluciones[i],
            'Energía': Egs[i],
            'Fidelidad': fidelity,
            'Error relativo': rel_err,
            'V-score': vsc
        })
    j += 1

df = pd.DataFrame(rows)

df.to_csv(file_path, index=False)
